<a href="https://colab.research.google.com/github/nashrah-18/YourIndianTranslatorupdated/blob/main/FinalEngToHin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Checking if GPU is running or not
!nvidia-smi

Fri Feb 21 06:15:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#LETS FIRST PUSH MY CREATED DATASET INTO MY HUGGINGFACE ACCOUNT
!pip install datasets huggingface_hub

In [3]:
#Load dataset locally
from datasets import load_dataset

dataset = load_dataset('json', data_files="/content/eng-hindi.json")

print(dataset['train'][4])

Generating train split: 0 examples [00:00, ? examples/s]

{'english_text': 'How are you?', 'hindi_translation': 'क्या हाल चाल है?', 'topic': 'Language Basics and Phrases'}


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
dataset.push_to_hub('nashrah18/indiantranslator')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/390 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/nashrah18/indiantranslator/commit/59395cd9f5d61210fc66be6f5e85f6bb69f2ea12', commit_message='Upload dataset', commit_description='', oid='59395cd9f5d61210fc66be6f5e85f6bb69f2ea12', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/nashrah18/indiantranslator', endpoint='https://huggingface.co', repo_type='dataset', repo_id='nashrah18/indiantranslator'), pr_revision=None, pr_num=None)

In [6]:
#DATA TOKENIZING AND MODEL TRAINING
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.7 MB/s eta 0:00:00


In [7]:
#Imports
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
#Choose the model
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [9]:
#Load our dataset that we just pushed so we can train it
raw_datasets = load_dataset("nashrah18/indiantranslator")

train-00000-of-00001.parquet:   0%|          | 0.00/15.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/236 [00:00<?, ? examples/s]

In [10]:
#Check the dataset
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['english_text', 'hindi_translation', 'topic'],
        num_rows: 236
    })
})

In [11]:
#Check specific column
raw_datasets['train'][100]

{'english_text': 'City',
 'hindi_translation': 'शहर',
 'topic': 'Language Basics and Phrases'}

In [12]:
#PREPROCESSING OUR DATA
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [13]:
#Check tokenizer
tokenizer("Hello, this is a sentence!")

{'input_ids': [12110, 2, 90, 23, 19, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
#Checking two sentences at the same time
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[12110, 2, 90, 23, 19, 8800, 61, 0], [239, 23, 414, 8800, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [15]:
#Check whether it does tokenizing to hindi sentences
with tokenizer.as_target_tokenizer():
    print(tokenizer(["यार, इंडिया में टूरिस्ट्स पर आम ऑनलाइन स्कैम्स कौन से हैं?"]))

{'input_ids': [[13005, 2, 28144, 11, 44122, 10972, 1307, 33, 2152, 4871, 18784, 16284, 1307, 391, 12, 28, 22, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
#Convert it into numbers
max_input_length=128
max_target_length=128

source_lang="english_text" #your dataset column name
target_lang="hindi_translation" #your dataset column name

def preprocess_function(examples):
    # Access the correct columns using column names
    inputs = examples[source_lang]
    targets = examples[target_lang]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[12110, 0], [172, 16, 213, 22, 0]], 'attention_mask': [[1, 1], [1, 1, 1, 1, 1]], 'labels': [[4788, 351, 26090, 0], [65, 1218, 1110, 22, 0]]}

In [18]:
#Map tokens
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

In [19]:
#Load model
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [40]:
batch_size = 16
learning_rate = 5e-5
weight_decay = 0.01
num_train_epochs = 15 #keep epochs high

In [41]:
#Passes data to your model batch wise
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [42]:
#Passes data with parameters
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [43]:
#Load Training data
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [44]:
#Compile your model
#Deploy your optimizer
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [45]:
#Feed my dataset to my model
model.fit(train_dataset, epochs=15)

Epoch 1/15
14/14 [==============================] - 26s 149ms/step - loss: 0.1215
Epoch 2/15
14/14 [==============================] - 2s 145ms/step - loss: 0.1241
Epoch 3/15
14/14 [==============================] - 2s 175ms/step - loss: 0.0992
Epoch 4/15
14/14 [==============================] - 2s 148ms/step - loss: 0.0956
Epoch 5/15
14/14 [==============================] - 2s 147ms/step - loss: 0.0818
Epoch 6/15
14/14 [==============================] - 2s 146ms/step - loss: 0.0667
Epoch 7/15
14/14 [==============================] - 2s 149ms/step - loss: 0.0576
Epoch 8/15
14/14 [==============================] - 2s 180ms/step - loss: 0.0519
Epoch 9/15
14/14 [==============================] - 3s 197ms/step - loss: 0.0397
Epoch 10/15
14/14 [==============================] - 3s 181ms/step - loss: 0.0373
Epoch 11/15
14/14 [==============================] - 3s 184ms/step - loss: 0.0366
Epoch 12/15
14/14 [==============================] - 3s 202ms/step - loss: 0.0258
Epoch 13/15
14/14 [=====

In [46]:
#Save the model
model.save_pretrained("indian_translatorv1")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


In [47]:
#Tokenize model
tokenizer.save_pretrained("indian_translatorv1")

('indian_translatorv1/tokenizer_config.json',
 'indian_translatorv1/special_tokens_map.json',
 'indian_translatorv1/vocab.json',
 'indian_translatorv1/source.spm',
 'indian_translatorv1/target.spm',
 'indian_translatorv1/added_tokens.json')

In [48]:
#Push our model indian_translatorv1
model.push_to_hub("nashrah18/indian_translatorv1")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

In [49]:
#Tokenize our model
tokenizer.push_to_hub("nashrah18/indian_translatorv1")

README.md:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/nashrah18/indian_translatorv1/commit/4e9159e8f87a0ee54fd034072aa15164e434fe57', commit_message='Upload tokenizer', commit_description='', oid='4e9159e8f87a0ee54fd034072aa15164e434fe57', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nashrah18/indian_translatorv1', endpoint='https://huggingface.co', repo_type='model', repo_id='nashrah18/indian_translatorv1'), pr_revision=None, pr_num=None)

#MODEL TESTING

In [50]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("indian_translatorv1")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at indian_translatorv1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [52]:
#Tokenize output
input_text  = "Brother"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
#Change numbers into text
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

भाईसाहब


In [70]:
from IPython.display import HTML, display
from ipywidgets import interact, widgets

def interactive_translation(text_input):
    if text_input:
        inputs = tokenizer([text_input], return_tensors='np')
        out = model.generate(**inputs, max_length=128)
        hindi_text = tokenizer.decode(out[0], skip_special_tokens=True)
        display(HTML(f"""
            <div style='padding: 10px; border-radius: 5px;'>
                <b>English:</b> {text_input}<br>
                <b>Hindi:</b> {hindi_text}
            </div>
        """))

# Create interactive widget
interact(interactive_translation, text_input=widgets.Text(description='English:', placeholder='Enter text to translate'))

interactive(children=(Text(value='', description='English:', placeholder='Enter text to translate'), Output())…

<function __main__.interactive_translation(text_input)>